---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [132]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)
df.str.replace('(', '')
df.str.replace("'", "")
df.str.replace(')', '')
df.str.replace('"','')

0           03/25/93 Total time of visit (in minutes):\n
1                         6/18/85 Primary Care Doctor:\n
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                  7 on 9/27/75 Audit C Score Current:\n
4      2/6/96 sleep studyPain Treatment Pain Level (N...
5                      .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current su...
7      10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                           3/7/86 SOS-10 Total Score:\n
9               (4/10/71)Score-1Audit C Score Current:\n
10     (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                         4/09/75 SOS-10 Total Score:\n
12     8/01/98 Communication with referring physician...
13     1/26/72 Communication with referring physician...
14     5/24/1990 CPT Code: 90792: With medical servic...
15     1/25/2011 CPT Code: 90792: With medical servic...
16           4/12/82 Total time of visit (in minutes):\n
17          1; 10/13/1976 Audit

In [133]:
def date_sorter():
    
    # Your code here
#     t1 = df.
#1st pattern
    t1 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})')
    dates = t1.reset_index()
    
#6th pattern
    index_left = ~df.index.isin([x for x in dates['level_0']])
    t2 = df[index_left].str.extractall(r'(?P<month>\d{1,2})/(?P<year>\d{4})')
    t2['day'] = 1
    t2_index = t2.reset_index()
    t2_index
    dates = pd.concat([dates, t2_index])
    
#2nd and 4th pattern
    index_left = ~df.index.isin([x for x in dates['level_0']])
    t3 = df[index_left].str.extractall(r'(?P<month>[A-Za-z]{3,})[.]?[-\W](?P<day>\d{1,2})[a-z]{,2}[,]?(?P<extra>-| )(?P<year>\d{4})')
    t3.drop(['extra'], axis=1, inplace=True)
    t3_index = t3.reset_index()
    t3_index

    dates = pd.concat([dates, t3_index], axis=0)

# 3rd pattern
    index_left = ~df.index.isin([x for x in dates['level_0']])
    t4 = df[index_left].str.extractall(r'(?P<day>\d{1,2}) (?P<month>[A-Za-z]{3,})[.|,]? (?P<year>\d{4})')
    t4_index = t4.reset_index()
    t4_index
    dates = pd.concat([dates, t4_index])
    
# 5th pattern
    index_left = ~df.index.isin([x for x in dates['level_0']])
    t5 = df[index_left].str.extractall(r'(?P<month>[A-Z]{1}[a-z]{2,}) (?P<year>\d{4})')
    t5['day']=1
    t5_index = t5.reset_index()
    t5_index
    dates = pd.concat([dates, t5_index])
    
#7th pattern

    index_left = ~df.index.isin([x for x in dates['level_0']])
    t6 = df[index_left].str.extractall(r'(?P<year>\d{4})')
    t6['day']=1
    t6['month']=1
    t6_index = t6.reset_index()
    t6_index
    dates = pd.concat([dates, t6_index])
    dates
    
    mon_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    
    dates.replace({'month':mon_dict}, inplace= True)
    
    dates['month'] = dates['month'].apply(lambda x: str(x))
    dates['day'] = dates['day'].apply(lambda x: str(x))
    dates['year'] = dates['year'].apply(lambda x: str(x))
    dates['year'] = dates['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    
    dates['date'] = dates['month'] + '/' + dates['day'] + '/' + dates['year']
    
    dates = dates[dates.match!=1]
    
    dates['date'] = pd.to_datetime(dates['date'])
    
    dates.sort_values(by='date', inplace=True)
    dates['level_0']
    
    return pd.Series(list(dates['level_0']))

In [134]:
t1 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<day>\d{1,2})[/-](?P<year>\d{2,4})')
dates = t1.reset_index()
dates

,level_0,match,month,day,year
0,0,0,03,25,93
1,1,0,6,18,85
2,2,0,7,8,71
3,3,0,9,27,75
4,4,0,2,6,96
5,5,0,7,06,79
6,6,0,5,18,78
7,7,0,10,24,89
8,8,0,3,7,86
9,9,0,4,10,71


In [135]:
dates = dates[dates.match!=1]
len(dates)

125

In [136]:
#2nd and 4th pattern
index_left = ~df.index.isin([x for x in dates['level_0']])
t3 = df[index_left].str.extractall(r'(?P<month>[A-Za-z]{3,})[.]?[-\W](?P<day>\d{1,2})[a-z]{,2}[,]?(?P<extra>-| )(?P<year>\d{4})')
t3.drop(['extra'], axis=1, inplace=True)
t3_index = t3.reset_index()
t3_index
i = t3_index[t3_index.month == 'Age'].index
t3_index.drop(i, inplace=True)
dates = pd.concat([dates, t3_index], axis=0)
len(t3_index)

34

In [137]:
# 3rd pattern
index_left = ~df.index.isin([x for x in dates['level_0']])
t4 = df[index_left].str.extractall(r'(?P<day>\d{1,2}) (?P<month>[A-Za-z]{3,})[.|,]? (?P<year>\d{4})')
t4_index = t4.reset_index()
t4_index
dates = pd.concat([dates, t4_index])
len(t4_index)

69

In [138]:
# 5th pattern
index_left = ~df.index.isin([x for x in dates['level_0']])
t5 = df[index_left].str.extractall('(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4})')
t5['day']=1
t5_index = t5.reset_index()
t5_index
dates = pd.concat([dates, t5_index])
t5_index

,level_0,match,month,year,day
0,228,0,September,1985,1
1,229,0,June,2011,1
2,230,0,May,1986,1
3,231,0,May,2016,1
4,232,0,July,1977,1
5,233,0,July,1990,1
6,234,0,Jul,2003,1
7,235,0,Oct,2015,1
8,236,0,May,1995,1
9,237,0,February,1976,1


In [139]:
dates = dates[dates.month!='Since']
dates

,day,level_0,match,month,year
0,25,0,0,03,93
1,18,1,0,6,85
2,8,2,0,7,71
3,27,3,0,9,75
4,6,4,0,2,96
5,06,5,0,7,79
6,18,6,0,5,78
7,24,7,0,10,89
8,7,8,0,3,86
9,10,9,0,4,71


In [140]:
#6th pattern
index_left = ~df.index.isin([x for x in dates['level_0']])
t2 = df[index_left].str.extractall(r'(?P<month>\d{1,2})/(?P<year>\d{4})')
t2['day'] = 1
t2_index = t2.reset_index()
t2_index
dates = pd.concat([dates, t2_index])
len(t2_index)

112

In [141]:
#7th pattern

index_left = ~df.index.isin([x for x in dates['level_0']])
t6 = df[index_left].str.extractall(r'(?P<year>\d{4})')
t6['day']=1
t6['month']=1
t6_index = t6.reset_index()
t6_index
dates = pd.concat([dates, t6_index])
len(t6_index)

45

In [142]:
mon_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    
dates.replace({'month':mon_dict}, inplace= True)

dates['month'] = dates['month'].apply(lambda x: str(x))
dates['day'] = dates['day'].apply(lambda x: str(x))
dates['year'] = dates['year'].apply(lambda x: str(x))
dates['year'] = dates['year'].apply(lambda x: '19' + x if len(x) == 2 else x)

dates['date'] = dates['month'] + '/' + dates['day'] + '/' + dates['year']

dates = dates[dates.match!=1]

dates['date'] = pd.to_datetime(dates['date'])

dates.sort_values(by='date', inplace=True)
dates['level_0']

9        9
85      84
2        2
53      53
28      28
19     474
28     153
13      13
4      129
99      98
112    111
31     225
31      31
46     171
66     191
31     486
72     415
107    335
36      36
95     323
62     405
79     422
32     375
37     380
2      345
57      57
26     481
93     436
105    104
71     299
      ... 
26     220
15     243
14     208
14     139
92     320
40     383
25     480
16     244
58     286
88     431
51     279
4      198
8      463
38     381
23     366
96     439
27     255
58     401
20     475
29     257
27     152
7      235
9      464
25     253
84     427
3      231
16     141
61     186
36     161
70     413
Name: level_0, Length: 500, dtype: int64